In [13]:
# ======================================================================
# 0. INSTALACIÓN
# ======================================================================
!pip -q install earthengine-api gradio geopandas numpy matplotlib rasterio \
                pillow folium branca scipy scikit-learn pyarrow fastparquet

# ======================================================================
# 1. IMPORTAR LIBRERÍAS
# ======================================================================
import ee, os, io, base64, traceback, re, warnings, math, calendar
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

import rasterio, rasterio.warp
from rasterio.enums import Resampling
from rasterio import transform
from PIL import Image

import folium, branca, gradio as gr
from branca.element import Template, MacroElement
from google.colab import auth, drive

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

# ----------------------------------------------------------------------
# MONTAJE INICIAL DE GOOGLE DRIVE
# ----------------------------------------------------------------------
drive.mount('/content/drive')

# ======================================================================
# 1-bis. CONFIGURAR FUENTE PERSONALIZADA (Palatino) + Seaborn (estilo unificado)
# ======================================================================
font_path = "/content/drive/My Drive/fonts/palatino-linotype.ttf"
if not Path(font_path).exists():
    raise FileNotFoundError(f"No se encontró el archivo: {font_path}")

fm.fontManager.addfont(font_path)
font_name = fm.FontProperties(fname=font_path).get_name()

# Estilo global de figuras (ticks, tipografías, tamaños).
mpl.rcParams.update({
    "font.family":  "serif",
    "font.serif":   [font_name],
    "pdf.fonttype": 42,
    "ps.fonttype":  42,
    "axes.unicode_minus": False,
    "axes.titleweight": "semibold",
    "axes.labelsize": 12,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "figure.dpi": 120,
})

sns.set_theme(
    style="ticks",
    context="notebook",
    rc={
        "font.family": "serif",
        "font.serif":  [font_name],
        "axes.titleweight": "semibold",
        "figure.dpi": 120,
    },
)
# Paleta de colores consistente.
BASE_PALETTE = sns.color_palette("deep")
COLOR_ACCENT = BASE_PALETTE[7]   # para barras e histogramas + hexbin fill
COLOR_WET    = BASE_PALETTE[0]   # azul
COLOR_DRY    = BASE_PALETTE[3]   # rojo
COLOR_CURVE_W= BASE_PALETTE[0]
COLOR_CURVE_D= BASE_PALETTE[3]
COLOR_SCATTER= "#6e6e6e"         # gris

print(f"✔️ Fuente registrada y activa: {font_name}")

# ======================================================================
# 2. PARÁMETROS
# ======================================================================
SCALE_TARGET  = 500
EXPORT_FOLDER = 'MODIS_Composite'
MAX_TASKS     = 400

NDVI_RANGE = (-1.0, 1.0)
LST_RANGE  = (-50.0, 60.0)

FIRMS_COLLECTION = 'FIRMS'

NDVI_SENTINEL = -32768
LST_SENTINEL  = -32768
QC_FOLDER     = f"/content/drive/My Drive/{EXPORT_FOLDER}"

# ======================================================================
# 3. EARTH ENGINE
# ======================================================================
auth.authenticate_user()
ee.Initialize(project='ee-example')  #añadir Cloud Project

# ======================================================================
# 4. REGIÓN DE INTERÉS (Boyacá + Cundinamarca)
# ======================================================================
def _get_regions():
    fc = (ee.FeatureCollection("FAO/GAUL/2015/level1")
          .filter(ee.Filter.inList('ADM1_NAME', ['Cundinamarca', 'Boyaca'])))
    return fc, fc.union().geometry()

# ======================================================================
# 5-a. NDVI y 5-b. LST (colecciones)
# ======================================================================
# Carga NDVI MODIS; filtros temporales/espaciales y dtypes.
def _get_ndvi_comp(start, end, geom):
    def _scale(img):
        nd = img.select('NDVI').multiply(0.0001).rename('NDVI')
        nd = nd.updateMask(nd.gte(-1).And(nd.lte(1)))
        return ee.Image(nd.copyProperties(img, ['system:time_start']))
    return (ee.ImageCollection('MODIS/061/MOD13A1')
            .filterDate(start, end).filterBounds(geom).map(_scale))

# Carga LST MODIS; normaliza y asegura máscara.
def _get_lst_comp(start, end, geom):
    def _scale(img):
        return (img.multiply(0.1).rename('LST')
                   .copyProperties(img, ['system:time_start']))
    return (ee.ImageCollection('projects/sat-io/open-datasets/gap-filled-lst/gf_day_1km')
              .filterDate(start, end)
              .filterBounds(geom)
              .map(_scale))

# ======================================================================
# 5-c. HAZARD (MCD12Q1)
# ======================================================================
# HAZARD: peso por clase VT desde tabla determinista
HAZARD_WEIGHTS = { 0:0, 1:0.85, 2:0.6, 3:0.6, 4:0.6, 5:0.82, 6:0.72, 7:0.4,
                   8:0.8, 9:0.8, 10:0.5, 11:0.1, 12:0.35, 13:0.05, 14:0.48,
                   15:0.3, 16:0.12, 17:0 }
def _get_hazard_img(year, geom):
    lc = (ee.ImageCollection('MODIS/061/MCD12Q1')
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .first().select('LC_Type1'))
    keys, vals = list(HAZARD_WEIGHTS.keys()), list(HAZARD_WEIGHTS.values())
    return (lc.remap(keys, vals)
              .rename('HAZARD')
              .updateMask(lc.mask())
              .clip(geom))

# ======================================================================
# 5-c-bis. VT (clase de cobertura anual) desde MCD12Q1 · LC_Type1 (1..17)
# ======================================================================
def _get_vt_img(year:int, geom):
    """
    Devuelve la imagen anual LC_Type1 (1..17) para el año dado, recortada a geom.
    No aplica pesos; es la clase cruda (VT).
    """
    vt = (ee.ImageCollection('MODIS/061/MCD12Q1')
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .first()
          .select('LC_Type1')
          .clip(geom))
    vt = vt.toInt16().updateMask(vt.mask())
    return vt.rename('VT')

# ======================================================================
# 5-c-ter. Nombres oficiales GEE para LC_Type1 (MCD12Q1, esquema IGBP)
# ======================================================================
VT_CLASS_NAMES = {
    1:  "Evergreen Needleleaf Forest",
    2:  "Evergreen Broadleaf Forest",
    3:  "Deciduous Needleleaf Forest",
    4:  "Deciduous Broadleaf Forest",
    5:  "Mixed Forests",
    6:  "Closed Shrublands",
    7:  "Open Shrublands",
    8:  "Woody Savannas",
    9:  "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-Up",
    14: "Cropland/Natural Vegetation Mosaics",
    15: "Permanent Snow and Ice",
    16: "Barren",
    17: "Water Bodies"
}
VT_VALID = set(VT_CLASS_NAMES.keys())

# ======================================================================
# 5-c-quater. Tabla HAZARD por cobertura (editable en UI) + utilidades
# ======================================================================
HAZARD_TABLE_DEFAULT = pd.DataFrame({
    "Code":        [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
    "LandCover":   ["Evergreen Broadleaf Forest","Deciduous Broadleaf Forest","Mixed Forests",
                    "Closed Shrublands","Open Shrublands","Woody Savannas","Savannas",
                    "Grasslands","Permanent Wetlands","Croplands","Urban and Built-Up",
                    "Cropland/Natural Vegetation Mosaics","Permanent Snow and Ice","Barren","Water Bodies"],
    "FireDensity": [0.1288,0.0000,0.0000,0.1279,0.0000,0.2046,0.2032,0.3430,0.1943,1.8119,0.3716,1.1816,0.0000,0.0000,0.0000],
    "HazardIndex": [0.0711,0.0000,0.0000,0.0706,0.0000,0.1129,0.1122,0.1893,0.1072,1.0000,0.2051,0.6521,0.0000,0.0000,0.0000],
}).astype({"Code":"int32"})

def vt_table_to_map(df: pd.DataFrame) -> dict:
    """Convierte la tabla editable a dict {code: hazard in [0,1]}."""
    d = {}
    for _, r in df.iterrows():
        try:
            c = int(r["Code"])
            v = float(r["HazardIndex"])
            d[c] = float(np.clip(v, 0.0, 1.0))
        except Exception:
            continue
    for k in VT_CLASS_NAMES:
        d.setdefault(k, 0.0)
    return d


# ======================================================================
# 5-d. Exportar extremos — año único + paso 0.01
# ======================================================================
# Exportación estable (Parquet/CSV)
def _export_extreme_pairs_adaptive_fast(year,
                                        step=0.01,
                                        p_low=2,
                                        p_high=98,
                                        ndvi_lag_days=8,
                                        scale_m=SCALE_TARGET,
                                        add_coords=False):
    try:
        print(f"[{year}] Iniciando exportaciones (step={step})…")

        start, end = f"{year}-01-01", f"{year}-12-31"
        _, geom = _get_regions()
        nd_ic = _get_ndvi_comp(start, end, geom)
        ls_ic = _get_lst_comp(start, end, geom)

        diff_ms = int(ndvi_lag_days * 86_400_000)
        joined = ee.Join.saveBest('LST', 'tDiff').apply(
            nd_ic, ls_ic,
            ee.Filter.maxDifference(diff_ms,
                leftField='system:time_start',
                rightField='system:time_start'))

        def _stack(img):
            ls = ee.Image(img.get('LST')).select('LST')
            return (ee.Image(img).select('NDVI').addBands(ls)
                    .copyProperties(img, ['system:time_start']))
        pairs = ee.ImageCollection(joined.map(_stack))

        dec = len(str(step).split('.')[1]) if '.' in str(step) else 0
        n_bins = int(round((1.0 + 0.2) / step))
        bins = [(round(i*step - 0.2, dec), round((i+1)*step - 0.2, dec))
                for i in range(n_bins)]
        if bins[-1][1] < 1.0:
            bins[-1] = (bins[-1][0], round(1.0, dec))

        for lo_nd, hi_nd in bins:
            sub = pairs.map(lambda im: im.updateMask(
                im.select('NDVI').gte(lo_nd).And(im.select('NDVI').lt(hi_nd))))

            stats  = sub.select('LST').toBands().reduceRegion(
                        ee.Reducer.percentile([p_low, p_high]),
                        geom, scale_m, bestEffort=True, maxPixels=1e13)

            vals   = stats.values()
            lst_lo = ee.Number(vals.get(0))
            lst_hi = ee.Number(vals.get(1))

            def _to_tbl(im):
                m = im.select('LST').lte(lst_lo).Or(im.select('LST').gte(lst_hi))
                im2 = im.updateMask(m)
                date = ee.Date(im.get('system:time_start')).format('YYYY-MM-dd')
                bands = ['NDVI','LST']
                if add_coords:
                    im2 = im2.addBands(ee.Image.pixelLonLat()); bands+=['longitude','latitude']
                return (im2.select(bands)
                           .sample(region=geom, scale=scale_m, geometries=False)
                           .map(lambda f: f.set('date', date)))

            tbl = sub.map(_to_tbl).flatten()

            tag = (f'EXT_{year}_BIN{"m" if lo_nd<0 else "p"}'
                   f'{abs(lo_nd):.{dec}f}-{"m" if hi_nd<0 else "p"}'
                   f'{abs(hi_nd):.{dec}f}')
            ee.batch.Export.table.toDrive(
                collection=tbl, description=tag, folder=EXPORT_FOLDER,
                fileNamePrefix=tag, fileFormat='CSV').start()
            print(f"[{year}] ➜ Tarea iniciada: {tag}.csv")

        return f"[{year}] Todas las tareas fueron lanzadas."

    except Exception as e:
        msg = f"[{year}] Error general: {e}\n{traceback.format_exc(limit=1)}"
        print(msg); return msg

def export_extremes_single_year(year_sel):
    year = int(year_sel)
    if year < 2003 or year > 2020:
        return "Error: el año debe estar entre 2003 y 2020."
    return _export_extreme_pairs_adaptive_fast(year, step=0.01, add_coords=False)

# ======================================================================
# 5-f. CONCATENAR TODOS LOS CSV “EXT_*” DE UN AÑO
# ======================================================================
# Alineación temporal/espacial; preserva llaves y máscaras.
def merge_extreme_csv(year_sel):
    try:
        year = int(year_sel)
        folder = f'/content/drive/My Drive/{EXPORT_FOLDER}'
        pattern = re.compile(rf'^EXT_{year}_BIN.*\.csv$')
        parts = [os.path.join(folder, f) for f in os.listdir(folder)
                 if pattern.match(f)]
        if not parts:
            return f"No se encontraron archivos EXT_{year}_BIN*.csv"
        dfs = [pd.read_csv(p) for p in sorted(parts)]
        all_df = pd.concat(dfs, ignore_index=True)
        out_name = f'EXT_ALL_NDVI_LST_{year}.csv'
        out_path = os.path.join(folder, out_name)
        all_df.to_csv(out_path, index=False)
        if all_df.empty:
            return "Falló la concatenación: DataFrame vacío."
        for p in parts:
            os.remove(p)
        return (f"✅ {len(parts)} archivos ➜ {out_name} "
                f"({len(all_df)} filas). Partes eliminadas.")
    except Exception as e:
        return f"Error unificando CSV: {e}\n{traceback.format_exc(limit=1)}"

# ======================================================================
# 5-g. EXPORTAR SERIE DIARIA COMPLETA (NDVI/LST/FIRMS/VT)
# ======================================================================
def _get_ndvi_raw(start, end, geom):
    return (ee.ImageCollection('MODIS/061/MOD13A1')
            .filterDate(start, end)
            .filterBounds(geom)
            .select('NDVI'))

def _get_lst_raw(start, end, geom):
    return (ee.ImageCollection('projects/sat-io/open-datasets/gap-filled-lst/gf_day_1km')
            .filterDate(start, end)
            .filterBounds(geom))

def _firms_conf_for_date(date_str, nd_proj, geom):
    d0 = ee.Date(date_str)
    d1 = d0.advance(1, 'day')
    col = (ee.ImageCollection(FIRMS_COLLECTION)
           .filterDate(d0, d1)
           .filterBounds(geom)
           .select('confidence'))

    def _nonempty(ic):
        img = ic.max().toUint8()
        proj375 = ee.Projection('EPSG:4326').atScale(375)
        img375  = img.setDefaultProjection(proj375)
        img_aggr = (img375
                    .reduceResolution(reducer=ee.Reducer.max(), bestEffort=True, maxPixels=4096)
                    .reproject(nd_proj)
                    .unmask(0)
                    .clip(geom))
        return img_aggr.rename('CONF')

    return ee.Image(ee.Algorithms.If(
        col.size().gt(0),
        _nonempty(col),
        ee.Image(0).toUint8().rename('CONF').reproject(nd_proj).clip(geom)
    ))

def _export_full_year_cells_raw(year_sel, ndvi_lag_days=8):
    try:
        year = int(year_sel)
        if year < 2003 or year > 2020:
            return "Error: el año debe estar entre 2003 y 2020."

        _, geom   = _get_regions()
        start_y   = f"{year}-01-01"
        end_y     = f"{year}-12-31"
        nd_ic     = _get_ndvi_raw(start_y, end_y, geom)
        ls_ic     = _get_lst_raw (start_y, end_y, geom)
        vt_static = _get_vt_img(year, geom)  # ← VT anual (LC_Type1)

        # Emparejar LST con NDVI (Δ t)
        diff_ms = int(ndvi_lag_days * 86_400_000)
        joined = ee.Join.saveBest('NDVI_best', 'tDiff').apply(
            ls_ic, nd_ic,
            ee.Filter.maxDifference(
                diff_ms, leftField='system:time_start', rightField='system:time_start'
            )
        )

        # Exportar por mes
        for m in range(1, 12 + 1):
            m0 = ee.Date.fromYMD(year, m, 1)
            m1 = m0.advance(1, 'month')
            lst_m = ee.ImageCollection(joined).filterDate(m0, m1)

            def _per_day(ls_img):
                nd_img  = ee.Image(ls_img.get('NDVI_best')).select('NDVI')
                nd_proj = nd_img.projection()

                # Reproyectar a la malla NDVI para evitar remuestreo posterior
                lst_raw = ee.Image(ls_img).reproject(nd_proj).rename('LST_raw')
                vt_img  = vt_static.reproject(nd_proj).rename('VT')
                lonlat  = ee.Image.pixelLonLat().reproject(nd_proj)

                date_str = ee.Date(ls_img.get('system:time_start')).format('YYYY-MM-dd')
                conf     = _firms_conf_for_date(date_str, nd_proj, geom)

                nd_unm   = nd_img.unmask(NDVI_SENTINEL).rename('NDVI_raw')
                lst_unm  = lst_raw.unmask(LST_SENTINEL)
                conf_u8  = conf.unmask(0).toUint8().rename('CONF')
                vt_i16   = vt_img.unmask(0).toInt16().rename('VT')

                stack = (ee.Image.cat([nd_unm, lst_unm, conf_u8, vt_i16, lonlat])
                         .updateMask(ee.Image(1).reproject(nd_proj))
                         .clip(geom))

                tbl = stack.sample(
                    region=geom, projection=nd_proj, geometries=False
                ).map(lambda f: f.set('date', date_str))
                return tbl

            fc_month = ee.FeatureCollection(lst_m.map(_per_day)).flatten()

            tag = f"FULL_{year}_{str(m).zfill(2)}_CELLS_RAW"
            ee.batch.Export.table.toDrive(
                collection     = fc_month,
                description    = tag,
                folder         = EXPORT_FOLDER,
                fileNamePrefix = tag,
                fileFormat     = 'CSV',
                selectors      = ['date','longitude','latitude','NDVI_raw','LST_raw','CONF','VT']
            ).start()

            print(f"[{year}] ➜ Tarea iniciada: {tag}.csv")

        return f"[{year}] Exportaciones mensuales iniciadas (12 CSV en '{EXPORT_FOLDER}')."

    except Exception as e:
        return f"Error exportando FULL RAW: {e}\n{traceback.format_exc(limit=1)}"


# ======================================================================
# 5-h. CSV ➜ PARQUET (BORRAR CSV)
# ======================================================================
def _convert_full_csvs_to_parquet(year_sel, compression='zstd'):
    try:
        year   = int(year_sel)
        folder = f"/content/drive/My Drive/{EXPORT_FOLDER}"
        pat    = re.compile(rf"^FULL_{year}_(\d{{2}})_CELLS_RAW\.csv$")
        files  = [f for f in os.listdir(folder) if pat.match(f)]
        if not files:
            return f"No hay CSV FULL_{year}_MM_CELLS_RAW.csv en {folder}"

        files.sort()
        total_rows, converted = 0, 0

        for fname in files:
            csv_path = os.path.join(folder, fname)
            pq_path  = os.path.join(folder, fname.replace(".csv", ".parquet"))

            df = pd.read_csv(csv_path, na_filter=True)

            if 'CONF' in df.columns:
                df['CONF'] = df['CONF'].fillna(0).astype('uint8')

            if 'NDVI_raw' in df.columns:
                try:
                    df['NDVI_raw'] = df['NDVI_raw'].astype('int16')
                except Exception:
                    df['NDVI_raw'] = df['NDVI_raw'].astype('int32')

            if 'LST_raw' in df.columns:
                try:
                    df['LST_raw'] = df['LST_raw'].astype('int16')
                except Exception:
                    df['LST_raw'] = df['LST_raw'].astype('int32')

            if 'VT' in df.columns:
                # LC_Type1 usa 1..17; 0 como “sin dato”
                try:
                    df['VT'] = df['VT'].fillna(0).astype('uint8')
                except Exception:
                    df['VT'] = df['VT'].fillna(0).astype('int16')

            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')

            df.to_parquet(pq_path, index=False, compression=compression)
            nrows = len(df)
            total_rows += nrows
            converted  += 1

            if os.path.exists(pq_path) and os.path.getsize(pq_path) > 0:
                os.remove(csv_path)
                print(f"✔️ {fname} ➜ {os.path.basename(pq_path)} ({nrows} filas) — CSV eliminado")
            else:
                print(f"⚠️ {fname}: Parquet no creado o vacío; CSV conservado")

        return (f"Convertidos {converted} CSV a Parquet (≈ {total_rows:,} filas). "
                f"Compresión='{compression}'. CSV originales eliminados.")

    except Exception as e:
        return f"Error en conversión CSV➜Parquet: {e}\n{traceback.format_exc(limit=1)}"

# ======================================================================
# 7. UTILIDADES PARA GEOTIFF NDVI–LST EXPORTADOS
# ======================================================================
def _tag2date(name):
    m = re.search(r'\d{8}', name)
    if not m:
        raise ValueError(f"No se encontró fecha en: {name}")
    return datetime.strptime(m.group(0), '%Y%m%d')

def _norm_ndvi(nd):  # ya en [-1,1]
    return nd

def _list_pairs(max_delta_days=8):
    folder = f'/content/drive/My Drive/{EXPORT_FOLDER}'
    if not os.path.isdir(folder):
        raise Exception("Aún no existe la carpeta de GeoTIFF en tu Drive.")
    nd_files = sorted(f for f in os.listdir(folder) if f.startswith('NDVI_'))
    ls_files = sorted(f for f in os.listdir(folder) if f.startswith('LST_'))
    haz_path = None
    ls_dict = {_tag2date(f): f for f in ls_files}
    pairs = []
    for nd_f in nd_files:
        d_nd = _tag2date(nd_f)
        d_closest = min(ls_dict, key=lambda d: abs((d - d_nd).days))
        if abs((d_closest - d_nd).days) <= max_delta_days:
            nd_path = os.path.join(folder, nd_f)
            ls_path = os.path.join(folder, ls_dict[d_closest])
            with rasterio.open(nd_path) as src:
                tr, crs = src.transform, src.crs
            pairs.append((nd_path, ls_path, tr, crs, haz_path))
    if not pairs:
        raise Exception("Sin pares NDVI–LST dentro de ±4 días.")
    return pairs

def _pair_by_date(date_str, max_delta_days=15):
    target = datetime.strptime(date_str, '%Y-%m-%d')
    pairs = _list_pairs(max_delta_days=max_delta_days)
    best = min(pairs, key=lambda p: abs((_tag2date(os.path.basename(p[0])) - target).days))
    d_best = _tag2date(os.path.basename(best[0]))
    if abs((d_best - target).days) > max_delta_days:
        raise RuntimeError(f"No hay NDVI dentro de ±{max_delta_days} días de {date_str}.")
    return best

_PAIRS = None
def get_pairs():
    global _PAIRS
    if _PAIRS is None:
        _PAIRS = _list_pairs()
    return _PAIRS

# ======================================================================
# 11. DENSIDAD NDVI–LST POR AÑO (HEXBIN + MARGINALES)
#     • Fuente: Parquet FULL_{YYYY}_MM_CELLS_RAW.parquet
# ======================================================================
def _list_full_parquets_for_year(year:int):
    pat = re.compile(rf"^FULL_{year}_(\d{{2}})_CELLS_RAW\.parquet$")
    files = [f for f in os.listdir(QC_FOLDER) if pat.match(f)]
    files.sort()
    return [os.path.join(QC_FOLDER, f) for f in files]

def _sample_year_points(year:int,
                        max_points:int = 400_000,
                        max_per_file:int = 40_000,
                        seed:int = 42):
    rng = np.random.default_rng(seed)
    paths = _list_full_parquets_for_year(year)
    if not paths:
        raise FileNotFoundError(
            f"No hay Parquet FULL_{year}_MM_CELLS_RAW.parquet en {QC_FOLDER}"
        )
    parts = []
    use_cols = ['NDVI_raw','LST_raw','latitude','longitude']
    for p in paths:
        df = pd.read_parquet(p, columns=use_cols)
        nd = df['NDVI_raw'].to_numpy()
        lr = df['LST_raw'].to_numpy()
        mask = (nd != NDVI_SENTINEL) & (lr != LST_SENTINEL)
        if not mask.any():
            continue
        ndv = np.clip(nd[mask].astype(np.float32) / 10000.0, -1.0, 1.0)
        lst = lr[mask].astype(np.float32) / 10.0
        lat = df.loc[mask, 'latitude'].to_numpy(dtype=np.float32)
        lon = df.loc[mask, 'longitude'].to_numpy(dtype=np.float32)

        m = (ndv >= -1.0) & (ndv <= 1.0) & (lst >= -20.0) & (lst <= 60.0)
        if not m.any():
            continue
        ndv, lst, lat, lon = ndv[m], lst[m], lat[m], lon[m]

        if len(ndv) > max_per_file:
            idx = rng.choice(len(ndv), size=max_per_file, replace=False)
            ndv, lst, lat, lon = ndv[idx], lst[idx], lat[idx], lon[idx]

        parts.append(pd.DataFrame({'NDVI': ndv, 'LST': lst, 'lat': lat, 'lon': lon}))
        del df

    if not parts:
        raise RuntimeError(f"Sin datos válidos tras filtrar en {year}.")
    out = pd.concat(parts, ignore_index=True)
    if len(out) > max_points:
        out = out.sample(n=max_points, random_state=seed)
    return out

def _apply_palatino_to_fig(fig, family):
    for ax in fig.axes:
        ax.title.set_fontfamily(family)
        ax.xaxis.label.set_fontfamily(family)
        ax.yaxis.label.set_fontfamily(family)
        for lab in ax.get_xticklabels() + ax.get_yticklabels():
            lab.set_fontfamily(family)

def _hexbin_year(year:int, mode:str="densidad", gridsize:int=45):
    """
    densidad  -> jointplot hexbin + marginales
    lat / lon -> scatter con marcador 'x' y barra de color (sin marginales)
    Estilo y colores consistentes Seaborn/Palatino.
    """
    df = _sample_year_points(year).replace([np.inf, -np.inf], np.nan).dropna(subset=["NDVI","LST"])
    if df.empty:
        fig, ax = plt.subplots(figsize=(6,2))
        ax.axis("off"); ax.text(.5,.5,f"Sin datos para {year}",ha="center",va="center")
        return fig

    xlim = (float(np.nanmin(df["NDVI"])), float(np.nanmax(df["NDVI"])))
    ylim = (float(np.nanmin(df["LST"])),  float(np.nanmax(df["LST"])))

    # --------- MODO DENSIDAD: HEXBIN + MARGINALES ----------
    if mode == "densidad":
        with sns.axes_style("ticks"), mpl.rc_context({"font.family": "serif", "font.serif": [font_name]}):
            g = sns.jointplot(
                data=df, x="NDVI", y="LST",
                kind="hex",
                color=COLOR_ACCENT,
                height=8.0, ratio=3, space=0,
                joint_kws=dict(gridsize=int(gridsize), mincnt=1),
                marginal_kws=dict(bins=50, color=COLOR_ACCENT, stat="count")
            )
            g.ax_joint.set_xlim(*xlim); g.ax_joint.set_ylim(*ylim)
            g.set_axis_labels("NDVI", "LST (°C)")

            g.ax_marg_x.cla()
            cnt_x, bins_x = np.histogram(df["NDVI"].to_numpy(), bins=50, range=xlim)
            centers_x = 0.5 * (bins_x[:-1] + bins_x[1:]); width_x = (bins_x[1] - bins_x[0]) * 0.9
            g.ax_marg_x.bar(centers_x, cnt_x, width=width_x, color=COLOR_ACCENT, edgecolor="none")
            g.ax_marg_x.set_xlim(*xlim); g.ax_marg_x.set_ylabel("Frecuencia")
            g.ax_marg_x.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=4, integer=True))
            g.ax_marg_x.ticklabel_format(style="sci", axis="y", scilimits=(0,0))
            g.ax_marg_x.tick_params(axis="y", labelleft=True)

            g.ax_marg_y.cla()
            cnt_y, bins_y = np.histogram(df["LST"].to_numpy(), bins=50, range=ylim)
            centers_y = 0.5 * (bins_y[:-1] + bins_y[1:]); height_y = (bins_y[1] - bins_y[0]) * 0.9
            g.ax_marg_y.barh(centers_y, cnt_y, height=height_y, color=COLOR_ACCENT, edgecolor="none")
            g.ax_marg_y.set_ylim(*ylim); g.ax_marg_y.set_xlabel("Frecuencia")
            g.ax_marg_y.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=4, integer=True))
            g.ax_marg_y.ticklabel_format(style="sci", axis="x", scilimits=(0,0))
            g.ax_marg_y.tick_params(axis="x", labelbottom=True)

            g.ax_joint.set_title(f"NDVI–LST · {year} · Densidad", pad=12)
            sns.despine(fig=g.fig); g.fig.tight_layout()

            for ax in g.fig.axes:
                ax.title.set_fontfamily(font_name)
                ax.xaxis.label.set_fontfamily(font_name)
                ax.yaxis.label.set_fontfamily(font_name)
                for lab in ax.get_xticklabels() + ax.get_yticklabels():
                    lab.set_fontfamily(font_name)
            return g.fig

    # --------- MODO LAT/LON: SCATTER 'X' + COLORBAR, SIN MARGINALES ----------
    var = "lat" if mode == "lat" else "lon"
    vals = df[var].to_numpy()
    try:
        cmap = sns.color_palette("cubehelix_r", as_cmap=True)
    except Exception:
        cmap = plt.cm.viridis
    norm = mpl.colors.Normalize(vmin=float(np.nanmin(vals)), vmax=float(np.nanmax(vals)))

    n = len(df)
    s = 6 if n > 200_000 else (8 if n > 100_000 else 12)

    with sns.axes_style("ticks"), mpl.rc_context({"font.family": "serif", "font.serif": [font_name]}):
        fig, ax = plt.subplots(figsize=(8.5, 6))
        sc = ax.scatter(df["NDVI"], df["LST"],
                        c=vals, cmap=cmap, norm=norm,
                        marker="x", s=s, linewidths=0.7, alpha=0.9)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim)
        ax.set_xlabel("NDVI"); ax.set_ylabel("LST (°C)")
        title = "Color = latitud" if mode == "lat" else "Color = longitud"
        ax.set_title(f"NDVI–LST · {year} · {title}", pad=12)
        ax.grid(alpha=.3)
        sns.despine(fig=fig)
        cbar = fig.colorbar(sc, ax=ax, pad=0.02)
        cbar.set_label("Latitud (°)" if mode == "lat" else "Longitud (°)")

        for a in fig.axes + [cbar.ax]:
            try: a.title.set_fontfamily(font_name)
            except Exception: pass
            if hasattr(a, "xaxis") and a.xaxis and a.xaxis.label: a.xaxis.label.set_fontfamily(font_name)
            if hasattr(a, "yaxis") and a.yaxis and a.yaxis.label: a.yaxis.label.set_fontfamily(font_name)
            for lab in a.get_xticklabels() + a.get_yticklabels():
                lab.set_fontfamily(font_name)

        fig.tight_layout()
        return fig

# ======================================================================
# 12-bis. ENVOLVENTES HÚMEDA/SECA (HISTGB + HUBER)
# ======================================================================
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import HuberRegressor

def _prep_xy_split_by_bin(df, step=0.01, n_min=5):
    d = df[['NDVI','LST']].replace([np.inf, -np.inf], np.nan).dropna().copy()
    d = d[(d['NDVI'] >= -1.0) & (d['NDVI'] <= 1.0)]
    d['bin'] = (np.floor(d['NDVI']/step)*step + step/2).round(3)

    Xw, yw, Xd, yd = [], [], [], []
    for b, g in d.groupby('bin'):
        s = np.sort(g['LST'].to_numpy(float))
        if s.size < 2:
            continue
        j = int(np.argmax(np.diff(s))) if s.size > 1 else 0
        lower, upper = s[:j+1], s[j+1:]
        if lower.size < n_min or upper.size < n_min:
            t = np.median(s); lower = s[s<=t]; upper = s[s>=t]
            if lower.size == 0 or upper.size == 0:
                continue
        nd_bin = float(b)
        Xw.append(np.full(lower.shape, nd_bin).reshape(-1,1));  yw.append(lower)
        Xd.append(np.full(upper.shape, nd_bin).reshape(-1,1));  yd.append(upper)

    if not Xw or not Xd:
        raise ValueError("No hay suficientes bins para separar bloques.")
    return (np.vstack(Xw), np.concatenate(yw)), (np.vstack(Xd), np.concatenate(yd))

def _fit_quantile_hgbr(X, y, q, monotone):
    model = HistGradientBoostingRegressor(
        loss="quantile", quantile=q, learning_rate=0.05, max_iter=600,
        min_samples_leaf=80, l2_regularization=1e-2, monotonic_cst=[monotone],
        random_state=42
    )
    model.fit(X, y)
    xg = np.linspace(-1.0, 1.0, 401).reshape(-1,1)
    yg = model.predict(xg)
    return xg.ravel(), yg

def _line_from_curve(xg, yg, fit_range=(0.15, 0.95)):
    mask = (xg >= fit_range[0]) & (xg <= fit_range[1])
    Xr = xg[mask].reshape(-1,1); yr = yg[mask]
    hr = HuberRegressor(alpha=1e-4).fit(Xr, yr)
    return float(hr.coef_[0]), float(hr.intercept_)

def envelope_curves_hgbr(df, q_wet=0.02, q_dry=0.98, step=0.01, n_min=5):
    (Xw, yw), (Xd, yd) = _prep_xy_split_by_bin(df, step=step, n_min=n_min)
    xg_w, wet_g = _fit_quantile_hgbr(Xw, yw, q=q_wet, monotone=+1)
    xg_d, dry_g = _fit_quantile_hgbr(Xd, yd, q=q_dry, monotone=-1)
    xg = np.linspace(-1.0, 1.0, 401)
    wet_curve = np.interp(xg, xg_w, wet_g)
    dry_curve = np.interp(xg, xg_d, dry_g)
    return xg, wet_curve, dry_curve

def _fit_edges_from_extremes(df,
                             q_wet=0.02, q_dry=0.98,
                             step=0.01, n_min=5,
                             fit_range=(0.15, 0.95)):
    xg, wet_g, dry_g = envelope_curves_hgbr(df, q_wet=q_wet, q_dry=q_dry,
                                            step=step, n_min=n_min)
    m_w, c_w = _line_from_curve(xg, wet_g, fit_range=fit_range)
    m_d, c_d = _line_from_curve(xg, dry_g, fit_range=fit_range)

    xm = xg[(xg>=fit_range[0]) & (xg<=fit_range[1])].mean()
    if m_w < 0:
        m_w = abs(m_w); c_w = float(np.mean(wet_g[(xg>=fit_range[0])&(xg<=fit_range[1])])) - m_w*xm
    if m_d >= 0:
        m_d = -abs(m_d) if m_d != 0 else -1e-6
        c_d = float(np.mean(dry_g[(xg>=fit_range[0])&(xg<=fit_range[1])])) - m_d*xm

    return (m_w, c_w), (m_d, c_d)

def _make_year_scatter_figure(df, year, m_w, c_w, m_d, c_d,
                              sample_max=200_000,
                              show_curves=True,
                              xlim=(-0.2, 1.0),
                              ylim=(-20, 60)):
    df = df[['NDVI','LST']].replace([np.inf, -np.inf], np.nan).dropna()
    df = df[(df['NDVI'] >= xlim[0]) & (df['NDVI'] <= xlim[1])]
    n = len(df)
    if n > sample_max:
        rng = np.random.default_rng(42)
        dfp = df.iloc[rng.choice(n, sample_max, replace=False)]
    else:
        dfp = df

    with sns.axes_style("ticks"), mpl.rc_context({"font.family": "serif", "font.serif": [font_name]}):
        fig, ax = plt.subplots(figsize=(8, 5))
        if not dfp.empty:
            ax.scatter(dfp['NDVI'], dfp['LST'], s=4, alpha=0.25,
                       color=COLOR_SCATTER, edgecolors='none')

        xs = np.linspace(xlim[0], xlim[1], 200)
        yw = m_w*xs + c_w
        yd = m_d*xs + c_d
        ax.plot(xs, yw, color=COLOR_WET,  lw=2.6, label=f"Húmeda  y={m_w:.2f}·x+{c_w:.2f}")
        ax.plot(xs, yd, color=COLOR_DRY,  lw=2.6, label=f"Seca    y={m_d:.2f}·x+{c_d:.2f}")

        if show_curves:
            try:
                xg, wet_g, dry_g = envelope_curves_hgbr(df)
                mask = (xg >= xlim[0]) & (xg <= xlim[1])
                ax.plot(xg[mask], wet_g[mask], color=COLOR_CURVE_W, lw=3, alpha=.50, label='Húmeda (curva)')
                ax.plot(xg[mask], dry_g[mask], color=COLOR_CURVE_D, lw=3, alpha=.50, label='Seca (curva)')
            except Exception:
                pass

        ax.set_xlim(*xlim); ax.set_ylim(*ylim)
        ax.set_xlabel('NDVI'); ax.set_ylabel('LST (°C)')
        ax.set_title(f'NDVI–LST {year}: nube (gris) + rectas')
        ax.grid(alpha=.3); ax.legend(loc='best', fontsize=9)
        sns.despine(fig=fig)
        fig.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=150); plt.close(fig)
        buf.seek(0); img = Image.open(buf).convert('RGB'); arr = np.array(img); buf.close()
        return arr

def plot_multi_year_edges(edge_list, alpha=.40):
    if not edge_list:
        raise ValueError("edge_list vacío")

    with sns.axes_style("ticks"), mpl.rc_context({"font.family": "serif", "font.serif": [font_name]}):
        fig, ax_plot = plt.subplots(figsize=(9, 6))
        x = np.linspace(-1, 1, 200)

        m_ws, c_ws, m_ds, c_ds = [], [], [], []
        for ed in edge_list:
            y_w = ed['m_w']*x + ed['c_w']
            y_d = ed['m_d']*x + ed['c_d']
            ax_plot.plot(x, y_w, color=COLOR_WET, alpha=alpha)
            ax_plot.plot(x, y_d, color=COLOR_DRY, alpha=alpha)
            m_ws.append(ed['m_w']); c_ws.append(ed['c_w'])
            m_ds.append(ed['m_d']); c_ds.append(ed['c_d'])

        m_wm, c_wm = np.mean(m_ws), np.mean(c_ws)
        m_dm, c_dm = np.mean(m_ds), np.mean(c_ds)

        ax_plot.plot(x, m_wm*x + c_wm, color=COLOR_WET,  lw=3,
                     label=f"Húmeda ⌀  y = {m_wm:.2f}·x + {c_wm:.2f}")
        ax_plot.plot(x, m_dm*x + c_dm, color=COLOR_DRY,  lw=3,
                     label=f"Seca   ⌀  y = {m_dm:.2f}·x + {c_dm:.2f}")

        ax_plot.set_xlabel('NDVI'); ax_plot.set_ylabel('LST (°C)')
        ax_plot.set_title('Rectas húmeda y seca por año + promedio')
        ax_plot.grid(alpha=.3); ax_plot.legend(fontsize=9, loc='upper left')
        ax_plot.set_xlim(-1, 1)
        sns.despine(fig=fig)
        fig.tight_layout()

    header = ["Año", "a₁", "b₁", "a₂", "b₂"]
    rows = [[str(ed['year']),
             f"{ed['m_w']:.4f}", f"{ed['c_w']:.4f}",
             f"{ed['m_d']:.4f}", f"{ed['c_d']:.4f}"]
            for ed in edge_list]
    table_text = '\t'.join(header) + '\n' + '\n'.join(['\t'.join(row) for row in rows])
    return fig, table_text

# ======================================================================
# 13. Mapa con pesos calibrados usando VT desde Parquet diario
#      • Toma VT de FULL_YYYY_MM_CELLS_RAW.parquet (columna VT) del día indicado
#      • Rasteriza VT -> HAZARD usando la tabla editable (vt_table_df)
#      • Calcula TVDI con rectas a1,b1 (húmeda) y a2,b2 (seca)
#      • FDCI = (w1*LST_SER + w2*ND_DRY + w3*TVDI + w4*HAZ + 1)/4
# ======================================================================

def _parquet_path_for_date(date_str: str) -> str:
    """Devuelve la ruta del Parquet mensual correspondiente a date_str."""
    try:
        y = int(date_str[0:4])
        m = int(date_str[5:7])
    except Exception:
        raise ValueError(f"Fecha inválida: {date_str} (esperado AAAA-MM-DD)")
    fname = f"FULL_{y}_{str(m).zfill(2)}_CELLS_RAW.parquet"
    path  = os.path.join(QC_FOLDER, fname)
    if not os.path.exists(path):
        raise FileNotFoundError(f"No existe {fname} en {QC_FOLDER}")
    return path

def _load_vt_points_for_date(date_str: str):
    """
    Carga (lon, lat, vt) SOLO del día solicitado desde el parquet mensual.
    Devuelve tres arrays (float64, float64, int16). Filtra VT válidos (1..17).
    """
    path = _parquet_path_for_date(date_str)
    df = pd.read_parquet(path, columns=["date", "longitude", "latitude", "VT"])
    if not pd.api.types.is_object_dtype(df["date"]):
        df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.strftime("%Y-%m-%d")
    df = df[df["date"] == date_str].dropna(subset=["longitude", "latitude", "VT"])
    if df.empty:
        return np.array([]), np.array([]), np.array([], dtype=np.int16)
    vt_arr = pd.to_numeric(df["VT"], errors="coerce").astype("Int16").to_numpy()
    mask   = np.isin(vt_arr, list(VT_VALID))
    if not mask.any():
        return np.array([]), np.array([]), np.array([], dtype=np.int16)
    lon = df.loc[mask, "longitude"].to_numpy()
    lat = df.loc[mask, "latitude"].to_numpy()
    vt  = vt_arr[mask].astype(np.int16)
    return lon, lat, vt

def _rasterize_hazard_from_points(lon, lat, vt, vt_map, nd_tr, nd_crs, out_shape):
    """
    Crea raster HAZARD (float32) alineado a la malla NDVI/LST usando:
      - puntos (lon,lat,vt)
      - vt_map: dict {code: hazard[0..1]}
      - nd_tr, nd_crs: transform y CRS del NDVI
      - out_shape: (rows, cols)
    """
    hz = np.zeros(out_shape, dtype=np.float32)

    if lon.size == 0:
        return hz
    try:
        dst = nd_crs if isinstance(nd_crs, str) else nd_crs.to_string()
    except Exception:
        dst = nd_crs
    if dst and str(dst).upper() not in ("EPSG:4326", "OGC:CRS84", "WGS84"):
        xs, ys = rasterio.warp.transform("EPSG:4326", nd_crs, lon.tolist(), lat.tolist())
        xs = np.asarray(xs); ys = np.asarray(ys)
    else:
        xs, ys = lon, lat

    rr, cc = rasterio.transform.rowcol(nd_tr, xs, ys)
    rr = np.asarray(rr); cc = np.asarray(cc)

    rows, cols = out_shape
    inb = (rr >= 0) & (rr < rows) & (cc >= 0) & (cc < cols)
    if not inb.any():
        return hz

    rr = rr[inb]; cc = cc[inb]; vt = vt[inb]

    max_code = max(VT_VALID)
    lut = np.zeros(max_code + 1, dtype=np.float32)
    for k, v in vt_map.items():
        if 0 <= int(k) <= max_code:
            lut[int(k)] = float(np.clip(v, 0.0, 1.0))
    hz_vals = lut[np.clip(vt, 0, max_code)]

    hz[rr, cc] = hz_vals.astype(np.float32)
    return hz

def _map_from_parquet(date_map_str,
                      a1, b1, a2, b2,               # rectas húmeda/seca (NDVI en [-1,1])
                      w_lst, w_ndvi, w_tvdi, w_haz,  # pesos FDCI
                      vt_table_df,
                      add_firms_points=False,        # dibuja puntos CONF>0 desde parquet
                      grid_deg=0.0045,               # ~500 m en el ecuador
                      lst_norm_min=None,             # LST min (°C) para normalizar (None = auto p02)
                      lst_norm_max=None):            # LST max (°C) para normalizar (None = auto p98)
    """
    Genera un mapa FDCI a partir del Parquet diario.

    Correcciones clave:
      • Se filtran explícitamente los centinelas: NDVI_raw == -32768, LST_raw == -32768.
      • NDVI se usa crudo en [-1, 1] (no 0–1).
      • LST serie se normaliza con [min, max] provistos o, si no, con p02–p98 del DÍA,
        calculados DESPUÉS de filtrar centinelas y no finitos.
      • FIRMS: se pintan puntos con CONF>0 correspondientes a las filas válidas (tras m_ok).
    """
    import numpy as np, pandas as pd, folium
    from matplotlib import cm, colors

    # -------- 1) Cargar SOLO el día desde el Parquet del mes correspondiente --------
    path = _parquet_path_for_date(date_map_str)
    use_cols = ["date","longitude","latitude","NDVI","NDVI_raw",
                "LST","LST_raw","CONF","VT"]

    try:
        import pyarrow.parquet as pq
        cols_avail = set(pq.ParquetFile(path).schema.names)
        keep = [c for c in use_cols if c in cols_avail]
        df = pd.read_parquet(path, columns=keep)
    except Exception:
        df = pd.read_parquet(path)
        keep = [c for c in use_cols if c in df.columns]
        df = df[keep]

    if 'date' not in df.columns:
        return "<p style='color:red'><b>El parquet no tiene columna 'date'.</b></p>"

    if not pd.api.types.is_object_dtype(df["date"]):
        df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.strftime("%Y-%m-%d")
    df = df[df["date"] == date_map_str]

    req_cols = {"longitude","latitude","VT"}
    if not req_cols.issubset(df.columns):
        faltan = ", ".join(sorted(req_cols - set(df.columns)))
        return f"<p style='color:red'><b>Faltan columnas en el parquet: {faltan}</b></p>"
    df = df.dropna(subset=["longitude","latitude","VT"])
    if df.empty:
        return "<p style='color:red'><b>No hay filas para ese día en el Parquet.</b></p>"

    # -------- 2) NDVI (crudo [-1,1]) y LST (°C) con FILTRO de centinelas --------
    # NDVI
    if "NDVI_raw" in df.columns:
        nd_raw = pd.to_numeric(df["NDVI_raw"], errors="coerce").to_numpy(np.int32)
        mask_nd = nd_raw != NDVI_SENTINEL
        nd = nd_raw.astype(np.float32) / 10000.0
    elif "NDVI" in df.columns:
        nd = pd.to_numeric(df["NDVI"], errors="coerce").to_numpy(np.float32)
        mask_nd = np.isfinite(nd) & (nd >= -1.5) & (nd <= 1.5)
    else:
        return "<p style='color:red'><b>No se encontró NDVI ni NDVI_raw en el parquet.</b></p>"

    # LST
    if "LST_raw" in df.columns:
        ls_raw = pd.to_numeric(df["LST_raw"], errors="coerce").to_numpy(np.int32)
        mask_ls = ls_raw != LST_SENTINEL
        ls = ls_raw.astype(np.float32) / 10.0
    elif "LST" in df.columns:
        ls = pd.to_numeric(df["LST"], errors="coerce").to_numpy(np.float32)
        mask_ls = np.isfinite(ls) & (ls > -150.0) & (ls < 100.0)
    else:
        return "<p style='color:red'><b>No se encontró LST ni LST_raw en el parquet.</b></p>"

    lon = pd.to_numeric(df["longitude"], errors="coerce").to_numpy(np.float64)
    lat = pd.to_numeric(df["latitude"],  errors="coerce").to_numpy(np.float64)
    vt  = pd.to_numeric(df["VT"],        errors="coerce").to_numpy(np.int16)

    # Para FIRMS:
    conf_vals = None
    if "CONF" in df.columns:
        conf_vals = pd.to_numeric(df["CONF"], errors="coerce").to_numpy(np.float32)
    else:
        conf_vals = None

    # Máscara final: centinelas + finitos + VT válido
    m_ok = (mask_nd & mask_ls &
            np.isfinite(nd) & np.isfinite(ls) &
            np.isfinite(lon) & np.isfinite(lat) &
            (vt > 0))

    if not np.any(m_ok):
        return "<p style='color:red'><b>Sin datos válidos (centinelas/NaN/VT) tras filtrar.</b></p>"

    nd = np.clip(nd[m_ok], -1.0, 1.0)
    ls = ls[m_ok]
    lon = lon[m_ok]; lat = lat[m_ok]; vt = vt[m_ok]
    if conf_vals is not None:
        conf_vals = conf_vals[m_ok]  # alinear FIRMS con las filas válidas

    # -------- 3) Features por píxel --------
    # LST serie normalizada por límites provistos o p02–p98 del día
    if (lst_norm_min is not None) and (lst_norm_max is not None):
        try:
            lmin = float(lst_norm_min); lmax = float(lst_norm_max)
        except Exception:
            lmin, lmax = np.nan, np.nan
        if (not np.isfinite(lmin)) or (not np.isfinite(lmax)) or (lmax <= lmin):
            lmin, lmax = np.percentile(ls, [2, 98])
    else:
        lmin, lmax = np.percentile(ls, [2, 98])

    lst_ser = np.clip((ls - lmin) / max(1e-6, (lmax - lmin)), 0.0, 1.0)

    # NDVI crudo como feature ([-1,1])
    nd_feat = nd

    # TVDI con rectas sobre NDVI en [-1,1]
    a1, b1, a2, b2 = map(float, (a1, b1, a2, b2))
    lst_w = a1*nd_feat + b1
    lst_d = a2*nd_feat + b2
    span  = np.maximum(1e-6, lst_d - lst_w)
    tvdi  = np.clip((ls - lst_w) / span, 0.0, 1.0)

    # HAZARD desde VT (tabla editable)
    vt_map = vt_table_to_map(vt_table_df)
    haz = np.array([vt_map.get(int(c), 0.0) for c in vt], dtype=np.float32)
    haz = np.clip(haz, 0.0, 1.0)

    # FDCI final
    w_lst, w_ndvi, w_tvdi, w_haz = map(float, (w_lst, w_ndvi, w_tvdi, w_haz))
    fdci = np.clip((w_lst*lst_ser + w_ndvi*nd_feat + w_tvdi*tvdi + w_haz*haz + 1.0) / 4.0, 0.0, 1.0)

    # -------- 4) Rasterizar a rejilla WGS84 --------
    pad = 0.01
    min_lon, max_lon = float(lon.min()) - pad, float(lon.max()) + pad
    min_lat, max_lat = float(lat.min()) - pad, float(lat.max()) + pad
    ncols = int(np.ceil((max_lon - min_lon) / grid_deg))
    nrows = int(np.ceil((max_lat - min_lat) / grid_deg))
    if ncols <= 0 or nrows <= 0:
        return "<p style='color:red'><b>Extensión inválida para rasterizar.</b></p>"

    cc = ((lon - min_lon) / grid_deg).astype(np.int64)
    rr = ((max_lat - lat) / grid_deg).astype(np.int64)
    m  = (rr >= 0) & (rr < nrows) & (cc >= 0) & (cc < ncols)
    rr, cc, val = rr[m], cc[m], fdci[m]

    idx = rr * ncols + cc
    acc = np.bincount(idx, weights=val, minlength=nrows*ncols).astype(np.float32)
    cnt = np.bincount(idx,              minlength=nrows*ncols).astype(np.float32)
    grid = (acc / np.maximum(cnt, 1)).reshape(nrows, ncols)
    grid[cnt.reshape(nrows, ncols) == 0] = np.nan

    # -------- 5) Folium --------
    cmap = cm.get_cmap("turbo")
    norm = colors.Normalize(vmin=0.0, vmax=1.0)
    rgba = (cmap(norm(np.nan_to_num(grid, nan=0.0))) * 255).astype(np.uint8)
    rgba[..., 3] = (np.isfinite(grid)).astype(np.uint8) * 255

    fmap = folium.Map(location=[(min_lat+max_lat)/2.0, (min_lon+max_lon)/2.0],
                      zoom_start=7, tiles="cartodbpositron")
    bounds = [[min_lat, min_lon], [max_lat, max_lon]]
    folium.raster_layers.ImageOverlay(
        image=rgba, bounds=bounds, name=f"FDCI (Parquet {date_map_str})",
        opacity=0.85, mercator_project=False
    ).add_to(fmap)

    # ---- Puntos FIRMS (CONF>0) ----
    n_pts = 0
    if add_firms_points and (conf_vals is not None):
        mask_fire = np.isfinite(conf_vals) & (conf_vals > 0)
        if np.any(mask_fire):
            lo_fire = lon[mask_fire]; la_fire = lat[mask_fire]; cf_fire = conf_vals[mask_fire]
            # Colorear por nivel de confianza: baja ≤30, media 31–60, alta >60
            def _color(c):
                if c <= 30: return "#FFB37A"   # naranja claro
                if c <= 60: return "#FF7F0E"   # naranja
                return "#E62300"               # rojo
            for lo, la, c in zip(lo_fire, la_fire, cf_fire):
                folium.CircleMarker(
                    location=[float(la), float(lo)],
                    radius=2.2, color=_color(float(c)),
                    fill=True, fill_opacity=0.95, opacity=0.9, weight=0.7,
                    tooltip=f"CONF={float(c):.0f}"
                ).add_to(fmap)
            n_pts = int(mask_fire.sum())

    folium.LayerControl().add_to(fmap)

    # Cabecera con info de normalización y NDVI
    header = (f"<div style='font-family:{font_name}; font-size:13px; margin:6px 0;'>"
              f"<b>Normalización LST (°C)</b>: min={lmin:.2f} · max={lmax:.2f} &nbsp;|&nbsp; "
              f"<b>NDVI</b>: sin normalizar [-1, 1] &nbsp;|&nbsp; "
              f"<b>FIRMS</b>: {n_pts} puntos con CONF&gt;0 (filtrados con datos válidos)</div>")
    return header + fmap._repr_html_()

# ======================================================================
# 15. HISTOGRAMAS MULTIANUALES
# ======================================================================
NDVI_BINS_F = np.linspace(0.0, 1.0, 81)     # paso 0.025
LST_BINS_F  = np.arange(-20, 60 + 1, 1.0)   # -20..60 °C
CONF_BINS_F = np.arange(0, 100 + 5, 5.0)    # 0..100, paso 5

def _accum_year_hist_and_stats(year:int):
    paths = _list_full_parquets_for_year(year)
    if not paths:
        raise FileNotFoundError(f"No hay Parquet FULL_{year}_MM_CELLS_RAW.parquet en {QC_FOLDER}")

    ndvi_counts = np.zeros(len(NDVI_BINS_F)-1, dtype=np.int64)
    lst_counts  = np.zeros(len(LST_BINS_F)-1,  dtype=np.int64)
    conf_counts = np.zeros(len(CONF_BINS_F)-1, dtype=np.int64)

    total_rows = 0
    ndvi_miss = 0
    lst_miss  = 0
    conf_pos  = 0

    by_day_count = {}
    use_cols = ['date','longitude','latitude','NDVI_raw','LST_raw','CONF']
    for p in paths:
        df = pd.read_parquet(p, columns=use_cols)
        if not pd.api.types.is_datetime64_any_dtype(df['date']):
            df['date'] = pd.to_datetime(df['date'], errors='coerce')

        total_rows += len(df)
        vc = df['date'].dt.date.value_counts(dropna=True)
        for d, c in vc.items():
            by_day_count[d] = by_day_count.get(d, 0) + int(c)

        nd = df['NDVI_raw'].to_numpy()
        ndvi_miss += int((nd == NDVI_SENTINEL).sum())
        mask_nd = (nd != NDVI_SENTINEL)
        if mask_nd.any():
            ndv = (nd[mask_nd].astype(np.float32) / 10000.0)
            ndv = np.clip(ndv, -1.0, 1.0)
            h, _ = np.histogram(ndv, bins=NDVI_BINS_F)
            ndvi_counts += h

        lr = df['LST_raw'].to_numpy()
        lst_miss += int((lr == LST_SENTINEL).sum())
        mask_lr = (lr != LST_SENTINEL)
        if mask_lr.any():
            lstc = lr[mask_lr].astype(np.float32) / 10.0
            h, _ = np.histogram(lstc, bins=LST_BINS_F)
            lst_counts += h

        cf = df['CONF'].to_numpy()
        conf_pos += int((cf > 0).sum())
        mconf = (cf > 0) & (cf <= 100)
        if mconf.any():
            h, _ = np.histogram(cf[mconf].astype(np.float32), bins=CONF_BINS_F)
            conf_counts += h
        del df

    days_obs = len(by_day_count)
    days_exp = 366 if calendar.isleap(year) else 365
    days_missing = max(days_exp - days_obs, 0)
    cells_per_day = np.array(list(by_day_count.values()), dtype=np.int64)
    cells_min = int(cells_per_day.min()) if days_obs else 0
    cells_max = int(cells_per_day.max()) if days_obs else 0
    cells_avg = float(cells_per_day.mean()) if days_obs else 0.0

    stats = {
        'year': year, 'files': len(paths), 'rows_total': int(total_rows),
        'days_expected': int(days_exp), 'days_observed': int(days_obs),
        'days_missing': int(days_missing), 'cells_day_min': cells_min,
        'cells_day_max': cells_max, 'cells_day_avg': cells_avg,
        'pct_ndvi_missing': (ndvi_miss / total_rows * 100.0) if total_rows else 0.0,
        'pct_lst_missing' : (lst_miss  / total_rows * 100.0) if total_rows else 0.0,
        'pct_conf_positive': (conf_pos  / total_rows * 100.0) if total_rows else 0.0,
    }
    hists = {'ndvi': ndvi_counts, 'lst': lst_counts, 'conf': conf_counts}
    return stats, hists

def _plot_hist_grid(years, all_hists, bins, title, xlabel, xlim=None,
                    outer_axes_only=True):
    from math import ceil
    n = len(years); ncols = 2; nrows = ceil(n / ncols)

    with sns.axes_style("ticks"), mpl.rc_context({"font.family": "serif", "font.serif": [font_name]}):
        fig, axes = plt.subplots(
            nrows, ncols, figsize=(12, 2.2*nrows),
            sharex=True, sharey=False, squeeze=False
        )
        centers = 0.5*(bins[:-1] + bins[1:]); width = (bins[1]-bins[0]) * 0.9

        for i, y in enumerate(years):
            r, c = divmod(i, ncols)
            ax = axes[r, c]
            counts = all_hists[y]
            ax.bar(centers, counts, width=width, align='center',
                   color=COLOR_ACCENT, edgecolor="none")
            ax.set_title(str(y))
            if xlim: ax.set_xlim(*xlim)
            ax.grid(alpha=.3)

            if outer_axes_only:
                if r < nrows - 1:
                    ax.set_xlabel("")
                    ax.tick_params(labelbottom=False)
                if c > 0:
                    ax.set_ylabel("")
                    ax.tick_params(labelleft=False)
                else:
                    ax.set_ylabel("Frecuencia")

        for j in range(n, nrows*ncols):
            r, c = divmod(j, ncols)
            axes[r, c].axis('off')

        if outer_axes_only:
            fig.supxlabel(xlabel)
            fig.supylabel("Frecuencia")

        fig.suptitle(title, y=0.995, fontsize=14, fontfamily=font_name)
        sns.despine(fig=fig)
        fig.tight_layout(rect=[0,0,1,0.97])
        return fig

def _run_compact_histograms(years_txt):
    years = [int(y.strip()) for y in years_txt.split(',') if y.strip()]
    if not years: raise ValueError("No se proporcionaron años.")
    years = sorted(years)

    ndvi_h, lst_h, conf_h = {}, {}, {}
    rows = []
    for y in years:
        stats, hists = _accum_year_hist_and_stats(y)
        rows.append(stats); ndvi_h[y] = hists['ndvi']; lst_h[y] = hists['lst']; conf_h[y] = hists['conf']

    fig_ndvi = _plot_hist_grid(years, ndvi_h, NDVI_BINS_F, "NDVI ([-1,1])", "NDVI")
    fig_lst  = _plot_hist_grid(years, lst_h,  LST_BINS_F,  "LST (°C)", "°C", xlim=(LST_BINS_F[0], LST_BINS_F[-1]))
    fig_conf = _plot_hist_grid(years, conf_h, CONF_BINS_F, "CONF (>0, bins de 5)", "Confianza")

    cols = ["year","files","rows_total","days_expected","days_observed","days_missing",
            "cells_day_min","cells_day_max","cells_day_avg",
            "pct_ndvi_missing","pct_lst_missing","pct_conf_positive"]
    summary_df = pd.DataFrame(rows)[cols]
    summary_df["cells_day_avg"] = summary_df["cells_day_avg"].round(0).astype("int64")
    for col in ["pct_ndvi_missing","pct_lst_missing","pct_conf_positive"]:
        summary_df[col] = summary_df[col].round(2)

    return fig_ndvi, fig_lst, fig_conf, summary_df

# ======================================================================
# 15-bis. Histogramas por VT con incendios (diarios y netos)
#         • Lee FULL_YYYY_MM_CELLS_RAW.parquet (usa columnas VT y CONF)
#         • "Diarios": cada celda-día con CONF>0 cuenta 1
#         • "Netos": corridas consecutivas (por celda) cuentan 1
#         • Opción: apilar por niveles de confianza (baja/med/alta)
# ======================================================================

def _conf_tier_series(conf: pd.Series) -> pd.Series:
    # Baja ≤30, Media 31–60, Alta >60
    return pd.cut(conf.astype('float32'),
                  bins=[0, 30, 60, 100],
                  labels=["baja","media","alta"],
                  include_lowest=True, right=True)

def _iter_event_rows_for_year(year:int):
    """Devuelve un DataFrame SOLO con filas de eventos (CONF>0) de todo el año,
    con columnas mínimas: ['date','lat_q','lon_q','VT','CONF']"""
    paths = _list_full_parquets_for_year(year)
    parts = []
    use_cols = ['date','longitude','latitude','VT','CONF']
    for p in paths:
        df = pd.read_parquet(p, columns=use_cols)
        if df.empty:
            continue
        # Filtro de eventos
        m = (df['CONF'] > 0)
        if not m.any():
            continue
        sub = df.loc[m, ['date','longitude','latitude','VT','CONF']].copy()
        # Normalizar tipos
        if not pd.api.types.is_datetime64_any_dtype(sub['date']):
            sub['date'] = pd.to_datetime(sub['date'], errors='coerce')
        sub = sub.dropna(subset=['date'])
        # Redondeo a rejilla
        sub['lat_q'] = sub['latitude'].round(5)
        sub['lon_q'] = sub['longitude'].round(5)
        # Limitar a clases VT válidas
        sub['VT'] = pd.to_numeric(sub['VT'], errors='coerce').astype('Int16')
        sub = sub[sub['VT'].isin(VT_VALID)]
        if not sub.empty:
            parts.append(sub[['date','lat_q','lon_q','VT','CONF']])
        del df, sub
    if not parts:
        return pd.DataFrame(columns=['date','lat_q','lon_q','VT','CONF'])
    events = pd.concat(parts, ignore_index=True)
    events.sort_values(['VT','lat_q','lon_q','date'], inplace=True)
    return events

def _count_daily_by_vt(events: pd.DataFrame, with_tiers: bool):
    """Cuenta celda-día por VT. Si with_tiers=True, apila por nivel de confianza."""
    if events.empty:
        cols = ['VT','total','baja','media','alta'] if with_tiers else ['VT','total']
        return pd.DataFrame(columns=cols)

    if with_tiers:
        events['tier'] = _conf_tier_series(events['CONF'])
        # tamaño por VT y tier
        t = events.groupby(['VT','tier'], observed=True).size().unstack('tier', fill_value=0)
        t = t.reindex(columns=['baja','media','alta'], fill_value=0)
        t['total'] = t.sum(axis=1)
        t = t.reset_index()
    else:
        t = (events.groupby('VT', observed=True).size()
                      .rename('total').reset_index())
    return t

def _count_net_by_vt(events: pd.DataFrame, with_tiers: bool):
    """Cuenta eventos netos (corridas de días consecutivos por celda) por VT.
       La confianza del evento es el MÁXIMO CONF dentro de la corrida."""
    if events.empty:
        cols = ['VT','total','baja','media','alta'] if with_tiers else ['VT','total']
        return pd.DataFrame(columns=cols)

    # Identificar corridas por (VT, celda)
    def _runs_for_group(g):
        # g: filas de una celda- VT, ordenadas por fecha
        d = g['date'].to_numpy()
        # Inicio de corrida si: diff>1 día o es la primera
        start = np.ones(len(d), dtype=bool)
        if len(d) > 1:
            delta = (pd.Series(d).diff().dt.days.to_numpy())
            start[1:] = (delta[1:] > 1) | np.isnan(delta[1:])
        # Label por corrida
        run_id = np.cumsum(start)
        out = g.copy()
        out['run_id'] = run_id
        return out

    grouped = events.groupby(['VT','lat_q','lon_q'], sort=False, observed=True, group_keys=False)
    runs_df = grouped.apply(_runs_for_group)

    # Confianza por corrida (máximo)
    agg = (runs_df.groupby(['VT','lat_q','lon_q','run_id'], observed=True)
                    .agg(CONF_max=('CONF','max'))
                    .reset_index())

    if with_tiers:
        agg['tier'] = _conf_tier_series(agg['CONF_max'])
        t = (agg.groupby(['VT','tier'], observed=True).size()
                 .unstack('tier', fill_value=0)
                 .reindex(columns=['baja','media','alta'], fill_value=0))
        t['total'] = t.sum(axis=1)
        t = t.reset_index()
    else:
        t = (agg.groupby('VT', observed=True).size()
                    .rename('total').reset_index())
    return t

def _format_vt_table(df_counts: pd.DataFrame, with_tiers: bool):
    df = df_counts.copy()
    df['VT_name'] = df['VT'].map(VT_CLASS_NAMES)
    if with_tiers:
        cols = ['VT','VT_name','total','baja','media','alta']
    else:
        cols = ['VT','VT_name','total']
    df = df[cols].sort_values('total', ascending=False)
    return df

# ======================================================================
# 15-ter. Plot de barras por VT (simple o apilado por confianza)
# ======================================================================
def _plot_vt_bars(df_vt: pd.DataFrame, title: str, show_conf: bool):
    if df_vt.empty:
        fig, ax = plt.subplots(figsize=(7, 2))
        ax.axis('off')
        ax.text(.5,.5,"Sin eventos con CONF>0 en los años seleccionados",
                ha='center', va='center', color='red')
        return fig

    with sns.axes_style("ticks"), mpl.rc_context({"font.family":"serif", "font.serif":[font_name]}):
        labels = [f"{int(v)} · {n}" for v, n in zip(df_vt['VT'], df_vt['VT_name'])]
        x = np.arange(len(labels))
        height = max(5, 0.4*len(labels))
        fig, ax = plt.subplots(figsize=(min(14, 1.1*len(labels)+6), height))

        if show_conf and all(c in df_vt.columns for c in ['baja','media','alta']):
            b1 = ax.bar(x, df_vt['baja'],  label="Conf. baja (≤30)")
            b2 = ax.bar(x, df_vt['media'], bottom=df_vt['baja'],  label="Conf. media (31–60)")
            b3 = ax.bar(x, df_vt['alta'],  bottom=(df_vt['baja']+df_vt['media']), label="Conf. alta (>60)")
            totals = df_vt['total']
        else:
            b1 = ax.bar(x, df_vt['total'], color=COLOR_ACCENT, edgecolor="none", label="Total")
            totals = df_vt['total']

        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=35, ha='right')
        ax.set_ylabel("Conteo")
        ax.set_title(title, pad=10)
        ax.grid(axis='y', alpha=.3)
        ax.margins(x=0.01)

        for idx, val in enumerate(totals):
            ax.text(x[idx], val + (0.01*totals.max()), f"{val:,}",
                    ha='center', va='bottom', fontsize=9, fontfamily=font_name)

        if show_conf:
            ax.legend(frameon=False, ncols=3, loc='upper right', fontsize=9)

        sns.despine(fig=fig)
        fig.tight_layout()
        return fig

# ======================================================================
# 15-quater. Ejecutar histogramas VT para varios años (diarios y netos)
# ======================================================================
def _run_vt_histograms(years_txt: str, show_conf: bool):
    years = [int(y.strip()) for y in years_txt.split(',') if y.strip()]
    if not years:
        raise ValueError("No se proporcionaron años.")
    years = sorted(years)

    # Agregar TODOS los eventos (CONF>0) de los años seleccionados
    all_events = []
    for y in years:
        ev_y = _iter_event_rows_for_year(y)
        if not ev_y.empty:
            all_events.append(ev_y)
    if not all_events:
        empty = pd.DataFrame(columns=['VT','VT_name','total','baja','media','alta'] if show_conf else ['VT','VT_name','total'])
        fig1 = _plot_vt_bars(empty, "VT con incendios · eventos diarios", show_conf)
        fig2 = _plot_vt_bars(empty, "VT con incendios · eventos netos (no consecutivos)", show_conf)
        return fig1, fig2

    events = pd.concat(all_events, ignore_index=True)

    daily_counts = _count_daily_by_vt(events, with_tiers=show_conf)
    net_counts   = _count_net_by_vt(events,   with_tiers=show_conf)

    daily_df = _format_vt_table(daily_counts, with_tiers=show_conf)
    net_df   = _format_vt_table(net_counts,   with_tiers=show_conf)

    rng = f"{years[0]}–{years[-1]}" if len(years) > 1 else f"{years[0]}"
    tit1 = f"VT con incendios · eventos diarios (CONF>0) · Años {rng}"
    tit2 = f"VT con incendios · eventos netos (no consecutivos) · Años {rng}"

    fig_daily = _plot_vt_bars(daily_df, tit1, show_conf=show_conf)
    fig_net   = _plot_vt_bars(net_df,   tit2, show_conf=show_conf)
    return fig_daily, fig_net

# ======================================================================
# 16. Calibración y Validación de pesos FDCI (w_lst, w_ndvi, w_tvdi, w_haz)
#      - Carga parquet FULL_YYYY_MM_CELLS_RAW.parquet
#      - PR-AUC como objetivo
# ======================================================================
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, roc_curve

def _years_from_text(txt: str) -> list[int]:
    return [int(y.strip()) for y in txt.split(",") if y.strip()]

def _iter_full_parquets(year: int):
    for p in _list_full_parquets_for_year(year):
        yield p

def _estimate_lst_percentiles(years: list[int], sample_per_file:int=5000, seed:int=42):
    """Estimación robusta de p02 y p98 de LST (°C) para normalizar la serie completa."""
    rng = np.random.default_rng(seed)
    sample = []
    for y in years:
        for p in _iter_full_parquets(y):
            try:
                df = pd.read_parquet(p, columns=["LST_raw"])
                arr = df["LST_raw"].to_numpy()
                m = (arr != LST_SENTINEL)
                vals = (arr[m].astype(np.float32)/10.0)
                if vals.size == 0:
                    continue
                if vals.size > sample_per_file:
                    idx = rng.choice(vals.size, sample_per_file, replace=False)
                    vals = vals[idx]
                sample.append(vals)
            except Exception:
                continue
    if not sample:
        return -5.0, 45.0
    vec = np.concatenate(sample)
    p2, p98 = np.percentile(vec, [2, 98])
    if p98 <= p2:
        p2, p98 = np.min(vec), np.max(vec)
    return float(p2), float(p98)

def _build_dataset(years: list[int],
                   vt_map: dict,
                   a1: float, b1: float, a2: float, b2: float,
                   neg_pos_ratio: float = 3.0,
                   max_rows: int = 800_000,
                   seed: int = 42,
                   p2p98: tuple[float, float] | None = None):
    """
    Devuelve X (n,4): [LST_SER, NDVI, TVDI, HAZ] y y (n,).
    Si p2p98 se provee, usa esos límites globales (p02, p98) para normalizar LST_SER
    en lugar de estimarlos sobre 'years'.
    """
    rng = np.random.default_rng(seed)
    if p2p98 is None:
        p2, p98 = _estimate_lst_percentiles(years, seed=seed)
    else:
        p2, p98 = float(p2p98[0]), float(p2p98[1])

    feats, labels = [], []
    taken = 0
    for y in years:
        for path in _iter_full_parquets(y):
            use_cols = ["date","NDVI_raw","LST_raw","CONF","VT"]
            try:
                df = pd.read_parquet(path, columns=use_cols)
            except Exception:
                continue
            if df.empty:
                continue

            nd = df["NDVI_raw"].to_numpy()
            ls = df["LST_raw"].to_numpy()
            cf = df["CONF"].to_numpy()
            vt = pd.to_numeric(df["VT"], errors="coerce").to_numpy()
            m_valid = (nd != NDVI_SENTINEL) & (ls != LST_SENTINEL) & (~np.isnan(vt)) & (vt > 0)
            if not m_valid.any():
                continue

            nd = (nd[m_valid].astype(np.float32)/10000.0)   # [-1,1]
            ls = (ls[m_valid].astype(np.float32)/10.0)      # °C
            cf = (cf[m_valid].astype(np.float32))
            vt = (vt[m_valid].astype(np.int16))

            y_pos = (cf > 0).astype(np.uint8)
            n_pos = int(y_pos.sum())
            if n_pos == 0:
                continue

            idx_all = np.arange(len(y_pos))
            idx_pos = idx_all[y_pos == 1]
            idx_neg = idx_all[y_pos == 0]
            n_neg_take = min(int(neg_pos_ratio * n_pos), idx_neg.size)
            take_idx = np.concatenate([idx_pos, rng.choice(idx_neg, size=n_neg_take, replace=False)]) if n_neg_take > 0 else idx_pos

            nd = nd[take_idx]; ls = ls[take_idx]; vt = vt[take_idx]; y_lab = y_pos[take_idx]

            # 1) LST_SER con percentiles FIJOS
            lst_ser = (ls - p2) / max(1e-6, (p98 - p2))
            lst_ser = np.clip(lst_ser, 0.0, 1.0)

            # 2) NDVI crudo
            nd_feat = nd

            # 3) TVDI desde rectas
            lst_wet = a1*nd_feat + b1
            lst_dry = a2*nd_feat + b2
            span = np.maximum(1e-6, (lst_dry - lst_wet))
            tvdi = np.clip((ls - lst_wet) / span, 0.0, 1.0)

            # 4) HAZ desde VT
            haz = np.array([vt_map.get(int(c), 0.0) for c in vt], dtype=np.float32)
            haz = np.clip(haz, 0.0, 1.0)

            X_part = np.stack([lst_ser, nd_feat, tvdi, haz], axis=1)
            feats.append(X_part); labels.append(y_lab)

            if sum(x.shape[0] for x in feats) >= max_rows:
                break
        if sum(x.shape[0] for x in feats) >= max_rows:
            break

    if not feats:
        raise RuntimeError("No se pudo construir el dataset.")
    X = np.vstack(feats).astype(np.float32)
    y = np.concatenate(labels).astype(np.uint8)
    return X, y, (p2, p98)


# 2) USAR percentiles de TRAIN para ambos splits en _time_split_by_date
def _time_split_by_date(years_train: list[int], years_val: list[int],
                        vt_map: dict, a1,b1,a2,b2, neg_pos_ratio, max_rows, seed):
    # Estimar p02–p98 SOLO sobre TRAIN
    p2p98_train = _estimate_lst_percentiles(years_train, seed=seed)

    # Construir TRAIN y VALID usando los mismos límites
    Xtr, ytr, _ = _build_dataset(years_train, vt_map, a1,b1,a2,b2,
                                 neg_pos_ratio=neg_pos_ratio, max_rows=max_rows,
                                 seed=seed, p2p98=p2p98_train)
    Xva, yva, _ = _build_dataset(years_val,   vt_map, a1,b1,a2,b2,
                                 neg_pos_ratio=neg_pos_ratio, max_rows=max_rows,
                                 seed=seed+1, p2p98=p2p98_train)
    # Devolver los percentiles históricos
    return (Xtr, ytr, p2p98_train), (Xva, yva, p2p98_train)

def _fdci_from_X(X: np.ndarray, w: np.ndarray) -> np.ndarray:
    if w is None:
        w = np.ones(4, dtype=np.float32)
    s = (X @ w.astype(np.float32) + 1.0) / 4.0
    s = np.nan_to_num(s, nan=0.0, posinf=1.0, neginf=0.0)
    return np.clip(s, 0.0, 1.0)

def _rand_search_weights(X: np.ndarray, y: np.ndarray,
                         n_candidates:int=200, seed:int=42,
                         bounds=(-1.0, 1.0), metric:str="pr"):
    rng = np.random.default_rng(seed)

    if np.unique(y).size < 2:
        return np.array([1,1,1,1], dtype=np.float32), float("nan")

    best_s, best_w = -np.inf, None
    cand = [np.array([1,1,1,1], dtype=np.float32),
            np.array([1,0.5,1,0.5], dtype=np.float32),
            np.array([0.7,0.7,0.7,0.7], dtype=np.float32)]
    for _ in range(max(0, n_candidates - len(cand))):
        cand.append(rng.uniform(bounds[0], bounds[1], size=4).astype(np.float32))

    for w in cand:
        s = _fdci_from_X(X, w)
        try:
            score = average_precision_score(y, s) if metric == "pr" else roc_auc_score(y, s)
        except Exception:
            score = -np.inf
        if np.isfinite(score) and score > best_s:
            best_s, best_w = score, w.copy()

    if best_w is None:
        best_w = np.array([1,1,1,1], dtype=np.float32)
    return best_w, float(best_s)

def _evaluate_scores(y_true: np.ndarray, scores: np.ndarray):
    roc = roc_auc_score(y_true, scores)
    pr  = average_precision_score(y_true, scores)

    # Umbral óptimo por F1
    P, R, T = precision_recall_curve(y_true, scores)
    f1 = np.where((P+R)>0, 2*P*R/(P+R), 0.0)
    i  = int(np.nanargmax(f1))
    thr = float(T[i-1]) if i>0 and i-1 < len(T) else 0.5
    return {"roc_auc": float(roc), "pr_auc": float(pr), "f1_best": float(np.max(f1)), "thr_opt": thr}

def calibrate_weights(years_train_txt: str,
                      years_val_txt: str,
                      vt_table_df: pd.DataFrame,
                      a1: float, b1: float, a2: float, b2: float,
                      neg_pos_ratio: float = 3.0,
                      max_rows: int = 800_000,
                      n_candidates: int = 300,
                      metric: str = "pr",
                      seed:int = 42):
    years_tr = _years_from_text(years_train_txt)
    years_va = _years_from_text(years_val_txt)
    if not years_tr or not years_va:
        return None, "Error: especifica años de train y validación.", None, None

    vt_map = vt_table_to_map(vt_table_df)
    (Xtr, ytr, p_train), (Xva, yva, p_valid) = _time_split_by_date(
        years_tr, years_va, vt_map, a1,b1,a2,b2,
        neg_pos_ratio, max_rows, seed
    )

    if np.unique(ytr).size < 2:
        return None, ("Error: el TRAIN no tiene 0s y 1s tras el muestreo. "
                      "Sube 'Negativos por positivo', amplía años TRAIN o incluye meses/días sin incendio."), None, None
    if np.unique(yva).size < 2:
        return None, ("Error: la VALID no tiene 0s y 1s. Asegura que haya días sin incendio en VALID."), None, None

    w_best, s_best = _rand_search_weights(Xtr, ytr, n_candidates=n_candidates, seed=seed, metric=metric)
    scr_tr = _evaluate_scores(ytr, _fdci_from_X(Xtr, w_best))
    scr_va = _evaluate_scores(yva, _fdci_from_X(Xva, w_best))

    scores_tr = _fdci_from_X(Xtr, w_best)
    scores_va = _fdci_from_X(Xva, w_best)
    fpr_tr, tpr_tr, _ = roc_curve(ytr, scores_tr)
    fpr_va, tpr_va, _ = roc_curve(yva, scores_va)
    auc_tr = roc_auc_score(ytr, scores_tr)
    auc_va = roc_auc_score(yva, scores_va)
    fig_roc_tr = _plot_roc_same_style(fpr_tr, tpr_tr, auc_tr, tag="(train)")
    fig_roc_va = _plot_roc_same_style(fpr_va, tpr_va, auc_va, tag="(valid)")

    summary = (
        f"Normalización LST (°C) por serie:\n"
        f"  TRAIN  p02={p_train[0]:.2f}, p98={p_train[1]:.2f}\n"
        f"  VALID  p02={p_valid[0]:.2f}, p98={p_valid[1]:.2f}\n\n"
        f"Pesos óptimos (train, métrica={metric.upper()}):\n"
        f"  w_lst={w_best[0]:+.3f} | w_ndvi={w_best[1]:+.3f} | w_tvdi={w_best[2]:+.3f} | w_haz={w_best[3]:+.3f}\n\n"
        f"Train → PR-AUC={scr_tr['pr_auc']:.3f} · ROC-AUC={scr_tr['roc_auc']:.3f} · F1*={scr_tr['f1_best']:.3f} @thr≈{scr_tr['thr_opt']:.2f}\n"
        f"Valid → PR-AUC={scr_va['pr_auc']:.3f} · ROC-AUC={scr_va['roc_auc']:.3f} · F1*={scr_va['f1_best']:.3f} @thr≈{scr_va['thr_opt']:.2f}"
    )
    return w_best, summary, scr_tr, scr_va, fig_roc_tr, fig_roc_va

def _plot_roc_same_style(fpr, tpr, auc_roc, tag: str = ""):
    fig, axr = plt.subplots(figsize=(6, 5))
    axr.plot(fpr, tpr, lw=2, color="navy", label=f"AUC = {auc_roc:.3f}")
    axr.plot([0, 1], [0, 1], "--", color="gray", label="aleatorio")
    axr.set_xlabel("FPR")
    axr.set_ylabel("TPR")
    ttl = "Curva ROC — presencia/no-presencia"
    if tag:
        ttl += f" {tag}"
    axr.set_title(ttl)
    axr.grid(ls=":")
    axr.legend()
    fig.tight_layout()
    return fig

# ======================================================================
# 14. INTERFAZ GRADIO – organización por pestañas
# ======================================================================
with gr.Blocks(title="MODIS 500 m — NDVI / LST / FDCI + HAZARD + FIRMS") as demo:
    gr.Markdown(r"""
### Flujo de trabajo
1) **Serie completa RAW anual (todas las celdas/día) + Parquet**
2) **Explorar: Densidad (hexbin) · Mapa**
3) **Extremos (0.01) y Rectas multi-año**
4) **Histogramas multianuales (compactos)**
""")

    with gr.Tabs():
        # ---------------------- TAB 1 · SERIE COMPLETA RAW -------------------------
        with gr.TabItem("1) Serie completa RAW + Parquet"):
            gr.Markdown("#### Serie completa anual (RAW enteros) — NDVI/LST/FIRMS/VT (12 CSV mensuales)")
            with gr.Row():
                year_full = gr.Number(2019, label="Año (2003–2020)", precision=0)
                lag_full  = gr.Number(16, label="Δ días NDVI vs LST (join)", precision=0)
                full_btn  = gr.Button("Exportar FULL")
            full_out = gr.Textbox(label="Estado exportación FULL")

            gr.Markdown("#### CSV ➜ Parquet (y eliminar CSV)")
            with gr.Row():
                year_pq = gr.Number(2019, label="Año a convertir", precision=0)
                pq_btn  = gr.Button("Convertir a Parquet")
            pq_out = gr.Textbox(label="Estado conversión")

        # ---------------------- TAB 2 · EXPLORAR (SOLO DENSIDAD) -------------------
        with gr.TabItem("2) Explorar (Densidad)"):
            gr.Markdown("#### Densidad NDVI–LST por año (hexbin + marginales)")
            with gr.Row():
                year_hex   = gr.Number(2019, label="Año (2003–2020)", precision=0)
                mode_hex   = gr.Radio(["densidad", "lat", "lon"], value="densidad", label="Color por")
                grid_hex   = gr.Slider(20, 120, value=50, step=2, label="Resolución (gridsize)")
                hex_btn    = gr.Button("Generar densidad")
            hex_fig = gr.Plot(label="Hexbin + marginales (Seaborn)")

            def _hex_wrapper(y, m, gsz):
                try:
                    return _hexbin_year(int(y), str(m), int(gsz))
                except Exception as e:
                    fig, ax = plt.subplots(figsize=(6, 2))
                    ax.axis('off'); ax.text(.5,.5,str(e),ha='center',va='center',color='red')
                    return fig

            hex_btn.click(_hex_wrapper, [year_hex, mode_hex, grid_hex], hex_fig)

        # ---------------------- TAB 3 · EXTREMOS & MULTI-AÑO -----------------------
        with gr.TabItem("3) Extremos (0.01) y Multi-año"):
            gr.Markdown("#### Exportación extremos NDVI-LST (step = 0.01)")
            with gr.Row():
                year_ext = gr.Number(2019, label="Año a procesar (2003–2020)", precision=0)
                ext_btn  = gr.Button("Exportar extremos")
            ext_out = gr.Textbox(label="Estado exportación extremos")

            gr.Markdown("#### Unificar CSV de extremos en uno solo")
            with gr.Row():
                year_merge = gr.Number(2019, label="Año a unificar", precision=0)
                merge_btn  = gr.Button("Unificar y limpiar CSV")
            merge_out = gr.Textbox(label="Estado de unificación")

            gr.Markdown("#### Rectas húmeda/seca multi-año (EXT_ALL_NDVI_LST_YYYY.csv)")
            with gr.Row():
                years_txt = gr.Textbox(
                    "2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020",
                    label="Años (separados por coma)")
                multi_btn = gr.Button("Generar rectas multi-año")
            multi_fig = gr.Plot(label="Rectas multi-año")
            multi_txt = gr.Textbox(label="Tabla de coeficientes", lines=20)
            multi_gallery = gr.Gallery(label="Por año: nube + rectas", show_label=True)

            def _multi_wrapper(yrs):
                try:
                    years = [int(y.strip()) for y in yrs.split(',') if y.strip()]
                    if not years:
                        raise ValueError("No se especificaron años.")
                    edge_list, gallery_imgs = [], []
                    for y in years:
                        path = f"/content/drive/My Drive/{EXPORT_FOLDER}/EXT_ALL_NDVI_LST_{y}.csv"
                        if not os.path.exists(path):
                            raise FileNotFoundError(f"No existe: {path}")
                        df_y = pd.read_csv(path)
                        (m_w, c_w), (m_d, c_d) = _fit_edges_from_extremes(df_y)
                        edge_list.append({'year': y, 'm_w': m_w, 'c_w': c_w, 'm_d': m_d, 'c_d': c_d})
                        img_arr = _make_year_scatter_figure(
                            df_y, y, m_w, c_w, m_d, c_d, sample_max=200_000, show_curves=True
                        )
                        gallery_imgs.append((img_arr, f"{y}"))
                    fig_all, table_txt = plot_multi_year_edges(edge_list)
                    return fig_all, table_txt, gallery_imgs
                except Exception as e:
                    fig, ax = plt.subplots(figsize=(6, 2))
                    ax.text(.5, .5, str(e), ha='center', va='center', color='red')
                    ax.axis('off'); return fig, str(e), []

        # ---------------------- TAB 4 · HISTOGRAMAS MULTIANUALES ------------------
        with gr.TabItem("4) Histogramas multianuales (compactos)"):
            gr.Markdown("### Histogramas multianuales (doble columna) y resumen por año")
            years_compact = gr.Textbox(
                "2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020",
                label="Años (separados por coma)"
            )
            compact_btn = gr.Button("Generar")

            fig_ndvi_all = gr.Plot(label="NDVI [-1,1] · todos los años")
            fig_lst_all  = gr.Plot(label="LST (°C) · todos los años")
            fig_conf_all = gr.Plot(label="CONF (>0) · todos los años")
            summary_tbl  = gr.Dataframe(label="Resumen por año",
                                        interactive=False,
                                        wrap=True,
                                        col_count=(12, "fixed"))

            def _compact_wrapper(yrs):
                try:
                    return _run_compact_histograms(yrs)
                except Exception as e:
                    fig, ax = plt.subplots(figsize=(6, 2))
                    ax.axis('off'); ax.text(.5,.5,str(e),ha='center',va='center',color='red')
                    err_df = pd.DataFrame({"error":[str(e)]})
                    return (fig, fig, fig, err_df)

            compact_btn.click(_compact_wrapper, [years_compact],
                              [fig_ndvi_all, fig_lst_all, fig_conf_all, summary_tbl])

            # ----- VT con incendios (diarios y netos) -----
            gr.Markdown("### VT con incendios · Histogramas por cobertura (LC_Type1)")
            with gr.Row():
                years_vt = gr.Textbox(
                    "2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020",
                    label="Años (separados por coma) — para VT"
                )
                show_conf_ck = gr.Checkbox(False, label="Mostrar confianza FIRMS (barras apiladas)")
                vt_btn = gr.Button("Generar VT con incendios")

            fig_vt_daily = gr.Plot(label="VT con incendios · eventos diarios (CONF>0)")
            fig_vt_net   = gr.Plot(label="VT con incendios · eventos netos (no consecutivos)")

            def _vt_wrapper(yrs, show_conf):
                try:
                    return _run_vt_histograms(yrs, bool(show_conf))
                except Exception as e:
                    fig, ax = plt.subplots(figsize=(6, 2))
                    ax.axis('off'); ax.text(.5,.5,str(e),ha='center',va='center',color='red')
                    return (fig, fig)

            vt_btn.click(_vt_wrapper, [years_vt, show_conf_ck], [fig_vt_daily, fig_vt_net])

        # ---------------------- TAB 5 · CALIBRACIÓN & VALIDACIÓN -----------------------
        with gr.TabItem("5) Calibración & Validación"):
            gr.Markdown("### Calibración de pesos FDCI (w_lst, w_ndvi, w_tvdi, w_haz) y validación")

            with gr.Row():
                years_tr_txt = gr.Textbox("2015,2016,2017,2018", label="Años TRAIN (coma)")
                years_va_txt = gr.Textbox("2019,2020",            label="Años VALID (coma)")

            with gr.Row():
                a1_in = gr.Number(8.06, label="a₁ (húmeda)")
                b1_in = gr.Number( 0.22, label="b₁ (húmeda)")
                a2_in = gr.Number(-11.41, label="a₂ (seca)")
                b2_in = gr.Number(48.03, label="b₂ (seca)")

            gr.Markdown("#### Tabla HAZARD por cobertura (edita **HazardIndex**). Doble click para cambiar.")
            vt_table = gr.Dataframe(value=HAZARD_TABLE_DEFAULT.copy(),
                                    headers=list(HAZARD_TABLE_DEFAULT.columns),
                                    datatype=["number","str","number","number"],
                                    row_count=(len(HAZARD_TABLE_DEFAULT), "fixed"),
                                    interactive=True)

            gr.Markdown("#### Pesos actuales (puedes fijarlos manualmente o dejar que el calibrador los busque)")
            with gr.Row():
                w_lst_in  = gr.Slider(-1.0, 1.0, value=+1.0, step=0.01, label="w_lst  (LST serie)")
                w_ndvi_in = gr.Slider(-1.0, 1.0, value=-0.5, step=0.01, label="w_ndvi (NDVI crudo)")
                w_tvdi_in = gr.Slider(-1.0, 1.0, value=+1.0, step=0.01, label="w_tvdi (TVDI)")
                w_haz_in  = gr.Slider(-1.0, 1.0, value=+1.0, step=0.01, label="w_haz  (HAZARD)")

            with gr.Row():
                neg_ratio = gr.Slider(0.0, 100.0, value=3.0, step=1.0, label="Negativos por positivo (ratio)")
                max_rows  = gr.Number(800_000, label="Máx. filas a usar", precision=0)
                n_cand    = gr.Number(300, label="Candidatos aleatorios", precision=0)
                metric_sel= gr.Radio(["pr","roc"], value="pr", label="Métrica objetivo (train)")
                seed_in   = gr.Number(42, label="Seed", precision=0)

            with gr.Row():
                btn_cal = gr.Button("🔧 Calibrar pesos")
                btn_val = gr.Button("✅ Validar con pesos actuales")
            out_txt = gr.Textbox(label="Resumen calibración/validación", lines=10)

            with gr.Row():
                roc_tr_plot = gr.Plot(label="ROC — TRAIN")
                roc_va_plot = gr.Plot(label="ROC — VALID")

            gr.Markdown("#### Mapa con pesos (elige LST min/max para normalizar)")
            with gr.Row():
                map_date5 = gr.Textbox("2019-08-15", label="Fecha mapa (AAAA-MM-DD)")
                lst_min5  = gr.Textbox("", label="LST min (°C) · vacío = auto p02")
                lst_max5  = gr.Textbox("", label="LST max (°C) · vacío = auto p98")
                add_firms5= gr.Checkbox(False, label="Añadir FIRMS VIIRS")
                btn_map5  = gr.Button("Mostrar mapa con pesos")
            map_html5 = gr.HTML(label="Mapa FDCI calibrado")

            # ---------- Callbacks ----------
            def do_calibrate(yrs_tr, yrs_va, vt_df, a1, b1, a2, b2, ratio, mx, nc, met, seed):
                try:
                    w_best, summary, scr_tr, scr_va, fig_tr, fig_va = calibrate_weights(
                        yrs_tr, yrs_va, vt_df, float(a1), float(b1), float(a2), float(b2),
                        neg_pos_ratio=float(ratio), max_rows=int(mx), n_candidates=int(nc),
                        metric=str(met), seed=int(seed)
                    )
                    if w_best is None:
                        return summary, gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update()
                    return summary, float(w_best[0]), float(w_best[1]), float(w_best[2]), float(w_best[3]), fig_tr, fig_va
                except Exception as e:
                    return f"Error calibrando: {e}", gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update()

            def do_validate(yrs_tr, yrs_va, vt_df, a1, b1, a2, b2, ratio, mx, wlst, wnd, wtv, whz, seed):
                try:
                    vt_map = vt_table_to_map(vt_df)
                    years_val = years_from_text(yrs_va)
                    Xv, yv, _ = build_dataset(years_val, vt_map, float(a1), float(b1), float(a2), float(b2),
                                              neg_pos_ratio=float(ratio), max_rows=int(mx), seed=int(seed)+9)
                    w = np.array([wlst, wnd, wtv, whz], dtype=np.float32)
                    scores = _fdci_from_X(Xv, w)
                    txt = (f"Validación con pesos actuales\n"
                          f"  w_lst={w[0]:.3f} | w_ndvi={w[1]:.3f} | w_tvdi={w[2]:.3f} | w_haz={w[3]:.3f}\n")
                    roc_auc = roc_auc_score(yv, scores)
                    pr_auc = average_precision_score(yv, scores)
                    fpr, tpr, thr = precision_recall_curve(yv, scores)
                    txt += f"Valid • PR-AUC={pr_auc:.3f} • ROC-AUC={roc_auc:.3f}\n"
                    fpr_v, tpr_v, _ = roc_curve(yv, scores)
                    fig_roc_v = _plot_roc_same_style(fpr_v, tpr_v, roc_auc, tag="(valid)")
                    return txt, fig_roc_v
                except Exception as e:
                    return f"Error validando: {e}", None

            def _do_map(date_str, a1,b1,a2,b2, wlst,wnd,wtv,whz, vt_df, lst_min_txt, lst_max_txt, add_f):
                lm = None if str(lst_min_txt).strip()=="" else lst_min_txt
                lx = None if str(lst_max_txt).strip()=="" else lst_max_txt
                return _map_from_parquet(date_str, a1,b1,a2,b2, wlst,wnd,wtv,whz, vt_df,
                                         add_firms_points=add_f, grid_deg=0.0045,
                                         lst_norm_min=lm, lst_norm_max=lx)

            btn_cal.click(
                do_calibrate,
                inputs=[years_tr_txt, years_va_txt, vt_table, a1_in, b1_in, a2_in, b2_in, neg_ratio, max_rows, n_cand, metric_sel, seed_in],
                outputs=[out_txt, w_lst_in, w_ndvi_in, w_tvdi_in, w_haz_in, roc_tr_plot, roc_va_plot]
            )

            btn_val.click(do_validate,
                          inputs=[years_tr_txt, years_va_txt, vt_table, a1_in,b1_in,a2_in,b2_in,
                                  neg_ratio, max_rows, w_lst_in, w_ndvi_in, w_tvdi_in, w_haz_in, seed_in],
                          outputs=[out_txt])

            btn_map5.click(
                _do_map,
                inputs=[map_date5, a1_in, b1_in, a2_in, b2_in,
                        w_lst_in, w_ndvi_in, w_tvdi_in, w_haz_in, vt_table, lst_min5, lst_max5, add_firms5],
                outputs=[map_html5]
            )

    # ---------------------- CONEXIONES GLOBALES ----------------------
    # Tab 1
    def _full_wrapper(y, lag): return _export_full_year_cells_raw(y, ndvi_lag_days=int(lag))
    full_btn.click(_full_wrapper, [year_full, lag_full], full_out)
    pq_btn.click(_convert_full_csvs_to_parquet, [year_pq], pq_out)

    # Tab 2
    def _hex_wrapper(y, m, gsz):
        try:
            return _hexbin_year(int(y), str(m), int(gsz))
        except Exception as e:
            fig, ax = plt.subplots(figsize=(6, 2))
            ax.axis('off'); ax.text(.5,.5,str(e),ha='center',va='center',color='red')
            return fig
    hex_btn.click(_hex_wrapper, [year_hex, mode_hex, grid_hex], hex_fig)

    # Tab 3
    ext_btn.click(export_extremes_single_year, [year_ext], ext_out)
    merge_btn.click(merge_extreme_csv, [year_merge], merge_out)
    multi_btn.click(_multi_wrapper, [years_txt], [multi_fig, multi_txt, multi_gallery])

demo.launch(share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔️ Fuente registrada y activa: Palatino Linotype
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a0f73b38b5115c497.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
